In [ ]:
# prompt: mount

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Vit directly on degrade

In [ ]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
from torchvision import transforms, datasets
from torchvision.models.vision_transformer import vit_b_32
from torch.utils.data import Dataset, DataLoader
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, precision_score, recall_score, f1_score, accuracy_score
import torch.nn.functional as F
from datetime import datetime
from PIL import Image
import os

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/AML-PROJECT/iris_degrade.csv')
df['ID'] = df['Label'].str.extract(r'(\d+)', expand=False)
id_counts = df['ID'].value_counts()
valid_ids = id_counts[id_counts >= 10].head(100).index
df = df[df['ID'].isin(valid_ids)]
train_df, val_df = train_test_split(df, test_size=0.2, stratify=df['ID'], random_state=42)
le = LabelEncoder()
train_df['encoded_label'] = le.fit_transform(train_df['Label'])
val_df['encoded_label'] = le.transform(val_df['Label'])

In [ ]:
class IrisDataset(Dataset):
    def __init__(self, df, transform=None):
        self.df = df
        self.transform = transform

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        path = self.df.iloc[idx]['Path']
        label = self.df.iloc[idx]['encoded_label']
        image = Image.open(path).convert('RGB')
        if self.transform:
            image = self.transform(image)
        return image, label

transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor()
])

train_dataset = IrisDataset(train_df, transform=transform)
val_dataset = IrisDataset(val_df, transform=transform)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32)

In [ ]:
class LabelSmoothingLoss(nn.Module):
    def __init__(self, classes, smoothing=0.1):
        super(LabelSmoothingLoss, self).__init__()
        self.confidence = 1.0 - smoothing
        self.smoothing = smoothing
        self.cls = classes

    def forward(self, pred, target):
        pred = pred.log_softmax(dim=-1)
        with torch.no_grad():
            true_dist = torch.zeros_like(pred)
            true_dist.fill_(self.smoothing / (self.cls - 1))
            true_dist.scatter_(1, target.data.unsqueeze(1), self.confidence)
        return torch.mean(torch.sum(-true_dist * pred, dim=-1))

class FocalLoss(nn.Module):
    def __init__(self, gamma=2):
        super(FocalLoss, self).__init__()
        self.gamma = gamma

    def forward(self, inputs, targets):
        logp = F.log_softmax(inputs, dim=1)
        p = torch.exp(logp)
        loss = F.nll_loss(((1 - p) ** self.gamma) * logp, targets)
        return loss

loss_functions = {
    "CrossEntropyLoss": nn.CrossEntropyLoss(),
    "LabelSmoothingLoss": LabelSmoothingLoss(classes=len(le.classes_)),
    "FocalLoss": FocalLoss()
}
optimizers = ["SGD", "Adam"]

results = []

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
from tqdm import tqdm
def train_model(loss_fn_name, optimizer_name):
    model = vit_b_32(pretrained=True)
    print("model loaded")

    model.heads.head = nn.Linear(model.heads.head.in_features, len(le.classes_))
    model.to(device)

    criterion = loss_functions[loss_fn_name]
    optimizer = torch.optim.SGD(model.parameters(), lr=0.001) if optimizer_name == 'SGD' else torch.optim.Adam(model.parameters(), lr=0.001)
    print(f"start training")
    model.train()
    for epoch in range(5):
      print(f"Epoch {epoch+1}")
      for images, labels in tqdm(train_loader, desc=f"Training Epoch {epoch+1}"):
          images, labels = images.to(device), labels.to(device)
          optimizer.zero_grad()
          outputs = model(images)
          loss = criterion(outputs, labels)
          loss.backward()
          optimizer.step()
    print(f"start eval")
    model.eval()
    all_preds, all_labels = [], []
    with torch.no_grad():
        for images, labels in val_loader:
            images = images.to(device)
            outputs = model(images)
            preds = torch.argmax(outputs, dim=1).cpu().numpy()
            all_preds.extend(preds)
            all_labels.extend(labels.numpy())

    accuracy = accuracy_score(all_labels, all_preds)
    f1 = f1_score(all_labels, all_preds, average='macro')
    precision = precision_score(all_labels, all_preds, average='macro')
    recall = recall_score(all_labels, all_preds, average='macro')
    timestamp = datetime.now().strftime('%Y%m%d-%H%M%S')

    save_dir = "/content/drive/MyDrive/AML-PROJECT/logistic/vit_models"
    os.makedirs(save_dir, exist_ok=True)

    model_path = os.path.join(
        save_dir,
        f"vit_b_32__{loss_fn_name}__{optimizer_name}__{timestamp}.pt"
    )

    torch.save(model.state_dict(), model_path)
    print(f"model saved")
    results.append({
        "variant": "vit_b_32",
        "loss_fn": loss_fn_name,
        "optimizer": optimizer_name,
        "epochs": 5,
        "accuracy": round(accuracy, 6),
        "f1": round(f1, 6),
        "precision": round(precision, 6),
        "recall": round(recall, 6),
        "timestamp": timestamp
    })

In [ ]:
for loss_name in loss_functions:
    for opt_name in optimizers:
        train_model(loss_name, opt_name)

/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ViT_B_32_Weights.IMAGENET1K_V1`. You can also use `weights=ViT_B_32_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/vit_b_32-d86f8d99.pth" to /root/.cache/torch/hub/checkpoints/vit_b_32-d86f8d99.pth
100%|██████████| 337M/337M [00:14<00:00, 23.6MB/s]


model loaded
start training
Epoch 1


Training Epoch 1: 100%|██████████| 50/50 [18:47<00:00, 22.54s/it]


Epoch 2


Training Epoch 2: 100%|██████████| 50/50 [00:14<00:00,  3.36it/s]


...

Epoch 5


Training Epoch 5: 100%|██████████| 50/50 [00:14<00:00,  3.37it/s]


start eval


/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


model saved


In [ ]:
summary_df = pd.DataFrame(results)
print("\nSUMMARY RESULTS")
print(summary_df.sort_values(by='accuracy', ascending=False).to_string(index=False))


=== SUMMARY RESULTS ===
 variant            loss_fn optimizer  epochs  accuracy       f1  precision   recall       timestamp
vit_b_32          FocalLoss       SGD       5    0.0900 0.061665   0.063986 0.107744 20250624-125713
vit_b_32 LabelSmoothingLoss       SGD       5    0.0725 0.047223   0.038371 0.083333 20250624-125431
vit_b_32   CrossEntropyLoss       SGD       5    0.0550 0.046787   0.049295 0.070833 20250624-125147
vit_b_32 LabelSmoothingLoss      Adam       5    0.0125 0.000557   0.000284 0.015385 20250624-125553
vit_b_32   CrossEntropyLoss      Adam       5    0.0025 0.000026   0.000013 0.005181 20250624-125311
vit_b_32          FocalLoss      Adam       5    0.0025 0.000026   0.000013 0.005181 20250624-125835


# Vit 32 on logistic and svm

In [ ]:
import os
import pandas as pd
import numpy as np
from PIL import Image
from tqdm.auto import tqdm
import torch
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as T
import timm
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
import joblib

class IrisImageDataset(Dataset):
    def __init__(self, df, transform):
        self.df = df.reset_index(drop=True)
        self.transform = transform

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        row = self.df.loc[idx]
        img_path = row['image_path']
        img = Image.open(img_path).convert('RGB')
        x = self.transform(img)
        y = row['label']
        return x, y

CSV_PATH = "/content/drive/MyDrive/AML-PROJECT/623final_all.csv"
OUTDIR = "/content/drive/MyDrive/AML-PROJECT/logistic/vit_models"
os.makedirs(OUTDIR, exist_ok=True)

df = pd.read_csv(CSV_PATH)
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
df['label_encoded'] = le.fit_transform(df['label'])

train_df, test_df = train_test_split(df, test_size=0.2, stratify=df['label_encoded'], random_state=42)

transform = T.Compose([
    T.Resize((224,224)),
    T.ToTensor(),
    T.Normalize(mean=[0.485,0.456,0.406], std=[0.229,0.224,0.225])
])

vit = timm.create_model('vit_base_patch32_224', pretrained=True)
vit.reset_classifier(0)
vit.eval()
vit.cuda()

def extract_features(df):
    ds = IrisImageDataset(df, transform)
    dl = DataLoader(ds, batch_size=16, shuffle=False, num_workers=2)
    feats, labels = [], []
    with torch.no_grad():
        for x, y in tqdm(dl):
            x = x.cuda()
            f = vit(x)
            feats.append(f.cpu().numpy())
            labels.append(np.array([le.transform([label])[0] for label in y]))
    return np.concatenate(feats), np.concatenate(labels)

print("Extracting train features...")
X_train, y_train = extract_features(train_df)
print("Extracting test features...")
X_test, y_test = extract_features(test_df)

classifiers = {
    'Logistic_L2': LogisticRegression(penalty='l2', max_iter=1000),
    'Logistic_L1': LogisticRegression(penalty='l1', solver='saga', max_iter=1000),
    'Logistic_ElasticNet': LogisticRegression(penalty='elasticnet', l1_ratio=0.5, solver='saga', max_iter=1000),
    'SVM_Linear': SVC(kernel='linear', probability=True),
    'SVM_RBF': SVC(kernel='rbf', probability=True),
    'SVM_Poly': SVC(kernel='poly', degree=3, probability=True)
}
results = []
for name, clf in classifiers.items():
    print(f"\nTraining {name}...")
    clf.fit(X_train, y_train)
    joblib.dump(clf, os.path.join(OUTDIR, f"{name}.joblib"))

    y_pred = clf.predict(X_test)
    acc = accuracy_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred, average='macro', zero_division=0)
    prec = precision_score(y_test, y_pred, average='macro', zero_division=0)
    rec = recall_score(y_test, y_pred, average='macro', zero_division=0)

    results.append({
        'variant': name,
        'accuracy': acc,
        'f1': f1,
        'precision': prec,
        'recall': rec
    })

print("\n=== SUMMARY RESULTS ===")
print(f"{'variant':20s} {'accuracy':>8s} {'f1':>8s} {'precision':>10s} {'recall':>8s}")
for r in results:
    print(f"{r['variant']:20s} "
          f"{r['accuracy']:.4f} {r['f1']:.4f} "
          f"{r['precision']:.4f} {r['recall']:.4f}")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


model.safetensors:   0%|          | 0.00/353M [00:00<?, ?B/s]

Extracting train features...


  0%|          | 0/100 [00:00<?, ?it/s]

Extracting test features...


...


Training Logistic_L2...

Training Logistic_L1...


/usr/local/lib/python3.11/dist-packages/sklearn/linear_model/_sag.py:348: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(



Training Logistic_ElasticNet...


/usr/local/lib/python3.11/dist-packages/sklearn/linear_model/_sag.py:348: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(



Training SVM_Linear...

Training SVM_RBF...

Training SVM_Poly...

=== SUMMARY RESULTS ===
variant              accuracy       f1  precision   recall
Logistic_L2          0.4500 0.4150 0.4373 0.4500
Logistic_L1          0.3225 0.2961 0.3131 0.3225
Logistic_ElasticNet  0.3525 0.3194 0.3286 0.3525
SVM_Linear           0.3700 0.3544 0.3923 0.3700
SVM_RBF              0.1750 0.1606 0.1857 0.1750
SVM_Poly             0.2200 0.2027 0.2250 0.2200


# VitB16 and SVm and logistic

In [ ]:
import torch
import torchvision.transforms as transforms
from torchvision.models import vit_b_16
from torch.utils.data import Dataset, DataLoader
import pandas as pd
import numpy as np
from PIL import Image
from sklearn.preprocessing import LabelEncoder
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
from sklearn.model_selection import train_test_split
from tqdm.auto import tqdm
import joblib
import os

class IrisDataset(Dataset):
    def __init__(self, df, transform):
        self.df = df.reset_index(drop=True)
        self.transform = transform

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        row = self.df.loc[idx]
        image = Image.open(row['image_path']).convert("RGB")
        label = row['label_encoded']
        return self.transform(image), label

df = pd.read_csv("/content/drive/MyDrive/AML-PROJECT/623final_all.csv")
le = LabelEncoder()
df['label_encoded'] = le.fit_transform(df['label'])

train_df, val_df = train_test_split(df, test_size=0.2, stratify=df['label_encoded'], random_state=42)
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

train_loader = DataLoader(IrisDataset(train_df, transform), batch_size=32, shuffle=False)
val_loader = DataLoader(IrisDataset(val_df, transform), batch_size=32, shuffle=False)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
vit = vit_b_16(pretrained=True)
vit.heads = torch.nn.Identity()
vit.eval().to(device)

def extract_features(loader):
    feats, labels = [], []
    with torch.no_grad():
        for imgs, lbls in tqdm(loader):
            imgs = imgs.to(device)
            output = vit(imgs)
            feats.append(output.cpu().numpy())
            labels.append(lbls.numpy())
    return np.vstack(feats), np.concatenate(labels)

print("Extracting ViT-B/16 features...")
X_train, y_train = extract_features(train_loader)
X_val, y_val = extract_features(val_loader)

models = {
    "Logistic_L2": LogisticRegression(penalty="l2", solver="lbfgs", max_iter=1000, multi_class="multinomial"),
    "SVM_Linear": SVC(C=10,kernel="linear", probability=True)
}

results = []
for name, model in models.items():
    print(f"\nTraining {name}...")
    model.fit(X_train, y_train)
    save_path = f"/content/drive/MyDrive/AML-PROJECT/logistic/for submission/{name}.joblib"
    joblib.dump(model, save_path)
    print(f"Model saved to {save_path}")
    y_pred = model.predict(X_val)

    acc = accuracy_score(y_val, y_pred)
    f1 = f1_score(y_val, y_pred, average="macro", zero_division=0)
    prec = precision_score(y_val, y_pred, average="macro", zero_division=0)
    rec = recall_score(y_val, y_pred, average="macro", zero_division=0)

    results.append({
        "variant": name,
        "accuracy": acc,
        "f1": f1,
        "precision": prec,
        "recall": rec
    })

print("\n=== SUMMARY RESULTS ===")
print(f"{'variant':15s} {'accuracy':>8s} {'f1':>8s} {'precision':>10s} {'recall':>8s}")
for r in results:
    print(f"{r['variant']:15s} {r['accuracy']:.4f} {r['f1']:.4f} {r['precision']:.4f} {r['recall']:.4f}")

/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ViT_B_16_Weights.IMAGENET1K_V1`. You can also use `weights=ViT_B_16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Extracting ViT-B/16 features...


  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/13 [00:00<?, ?it/s]


Training Logistic_L2...


/usr/local/lib/python3.11/dist-packages/sklearn/linear_model/_logistic.py:1247: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(


Model saved to /content/drive/MyDrive/AML-PROJECT/logistic/for submission/Logistic_L2.joblib

Training SVM_Linear...
Model saved to /content/drive/MyDrive/AML-PROJECT/logistic/for submission/SVM_Linear.joblib

=== SUMMARY RESULTS ===
variant         accuracy       f1  precision   recall
Logistic_L2     0.6375 0.6141 0.6662 0.6375
SVM_Linear      0.5250 0.5053 0.5521 0.5250


# Vit + SVM and logistic on degrade

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/AML-PROJECT/iris_degrade.csv')
df['ID'] = df['Label'].str.extract(r'(\d+)', expand=False)
id_counts = df['ID'].value_counts()
valid_ids = id_counts[id_counts >= 10].head(100).index
df = df[df['ID'].isin(valid_ids)]
train_df, val_df = train_test_split(df, test_size=0.2, stratify=df['ID'], random_state=42)
le = LabelEncoder()
train_df['encoded_label'] = le.fit_transform(train_df['Label'])
val_df['encoded_label'] = le.transform(val_df['Label'])

In [ ]:
class IrisDataset(Dataset):
    def __init__(self, df, transform):
        self.df = df.reset_index(drop=True)
        self.transform = transform

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        img = Image.open(self.df.loc[idx, 'Path']).convert("RGB")
        label = self.df.loc[idx, 'encoded_label']
        return self.transform(img), label

transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

train_loader = DataLoader(IrisDataset(train_df, transform), batch_size=32, shuffle=False)
val_loader = DataLoader(IrisDataset(val_df, transform), batch_size=32, shuffle=False)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
vit = vit_b_16(pretrained=True)
vit.heads = torch.nn.Identity()
vit.eval().to(device)

def extract_features(loader):
    feats, labels = [], []
    with torch.no_grad():
        for imgs, lbls in tqdm(loader):
            imgs = imgs.to(device)
            output = vit(imgs)
            feats.append(output.cpu().numpy())
            labels.append(lbls.numpy())
    return np.vstack(feats), np.concatenate(labels)

print("Extracting features...")
X_train, y_train = extract_features(train_loader)
X_val, y_val = extract_features(val_loader)

models = {
    'Logistic_L2': LogisticRegression(penalty='l2', solver='lbfgs', max_iter=1000, multi_class='multinomial'),
    'SVM_Linear': SVC(kernel='linear', probability=True)
}

results = []
for name, model in models.items():
    print(f"\nTraining {name}...")
    model.fit(X_train, y_train)
    save_path = f"/content/drive/MyDrive/AML-PROJECT/logistic/for submission/degrade{name}.joblib"
    joblib.dump(model, save_path)
    print(f"Model saved to {save_path}")
    y_pred = model.predict(X_val)

    acc = accuracy_score(y_val, y_pred)
    f1 = f1_score(y_val, y_pred, average='macro', zero_division=0)
    prec = precision_score(y_val, y_pred, average='macro', zero_division=0)
    rec = recall_score(y_val, y_pred, average='macro', zero_division=0)

    results.append({
        'variant': name,
        'accuracy': acc,
        'f1': f1,
        'precision': prec,
        'recall': rec
    })

print("\n=== SUMMARY RESULTS ===")
print(f"{'variant':15s} {'accuracy':>8s} {'f1':>8s} {'precision':>10s} {'recall':>8s}")
for r in results:
    print(f"{r['variant']:15s} {r['accuracy']:.4f} {r['f1']:.4f} {r['precision']:.4f} {r['recall']:.4f}")

/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ViT_B_16_Weights.IMAGENET1K_V1`. You can also use `weights=ViT_B_16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Extracting features...


  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/13 [00:00<?, ?it/s]


Training Logistic_L2...


/usr/local/lib/python3.11/dist-packages/sklearn/linear_model/_logistic.py:1247: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(


Model saved to /content/drive/MyDrive/AML-PROJECT/logistic/for submission/degradeLogistic_L2.joblib

Training SVM_Linear...
Model saved to /content/drive/MyDrive/AML-PROJECT/logistic/for submission/degradeSVM_Linear.joblib

=== SUMMARY RESULTS ===
variant         accuracy       f1  precision   recall
Logistic_L2     0.9000 0.8849 0.8979 0.9038
SVM_Linear      0.8600 0.8323 0.8538 0.8607


# clip-vit logistic svm

In [ ]:
import torch
from PIL import Image
import pandas as pd
import numpy as np
from tqdm.auto import tqdm
from transformers import CLIPProcessor, CLIPModel
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, f1_score,precision_score,recall_score
from sklearn.preprocessing import StandardScaler

data = [[1, -1, 2], [2, 0, 0], [0, 1, -1]]
scaler = StandardScaler()
scaled_data = scaler.fit_transform(data)
print(scaled_data)

df = pd.read_csv('/content/drive/MyDrive/AML-PROJECT/iris_degrade.csv')  # change to noisy.csv for second run
df['ID'] = df['Label'].str.extract(r'(\d+)', expand=False)
id_counts = df['ID'].value_counts()
valid_ids = id_counts[id_counts >= 10].head(100).index
df = df[df['ID'].isin(valid_ids)]

train_df, val_df = train_test_split(df, test_size=0.2, stratify=df['ID'], random_state=42)
le = LabelEncoder()
train_df['encoded_label'] = le.fit_transform(train_df['Label'])
val_df['encoded_label'] = le.transform(val_df['Label'])

device = 'cuda' if torch.cuda.is_available() else 'cpu'
clip_model = CLIPModel.from_pretrained("openai/clip-vit-base-patch16").to(device)
clip_processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch16")

def extract_features(df):
    feats, labels = [], []
    for _, row in tqdm(df.iterrows(), total=len(df)):
        img = Image.open(row['Path']).convert("RGB")
        inputs = clip_processor(images=img, return_tensors="pt").to(device)
        with torch.no_grad():
            vision_out = clip_model.vision_model(**inputs)
            pooled = clip_model.visual_projection(vision_out.pooler_output)
        feats.append(pooled.cpu().numpy().squeeze())
        labels.append(row['ID'])
    return np.stack(feats), np.array(labels)

X_train, y_train = extract_features(train_df)
X_val, y_val = extract_features(val_df)

clf = LogisticRegression(multi_class='multinomial', max_iter=1000)
clf.fit(X_train, y_train)
y_pred = clf.predict(X_val)
joblib.dump(clf, "/content/drive/MyDrive/AML-PROJECT/logistic/for submission/clip_logistic.joblib")
print("Logistic model saved.")
print("Accuracy:", accuracy_score(y_val, y_pred))
print("Macro F1 :", f1_score(y_val, y_pred, average='macro'))
print(precision_score(y_val, y_pred, average='macro'))
print(recall_score(y_val, y_pred, average='macro'))

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_val_scaled = scaler.transform(X_val)
svm = SVC(kernel='rbf', gamma='scale')
svm.fit(X_train_scaled, y_train)
y_pred = svm.predict(X_val_scaled)
joblib.dump(svm, "/content/drive/MyDrive/AML-PROJECT/logistic/for submission/clip_svm.joblib")
print("SVM saved.")
print("SVM")
# === Evaluation ===
print("Accuracy:", accuracy_score(y_val, y_pred))
print("Macro F1 :", f1_score(y_val, y_pred, average='macro'))
print("Precision:",precision_score(y_val, y_pred, average='macro'))
print("Recall:",recall_score(y_val, y_pred, average='macro'))

[[ 0.         -1.22474487  1.33630621]
 [ 1.22474487  0.         -0.26726124]
 [-1.22474487  1.22474487 -1.06904497]]


  0%|          | 0/1600 [00:00<?, ?it/s]

  0%|          | 0/400 [00:00<?, ?it/s]

/usr/local/lib/python3.11/dist-packages/sklearn/linear_model/_logistic.py:1247: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(


Logistic model saved.
Accuracy: 0.73
Macro F1 : 0.7184379509379508
0.7440714285714286
0.73


/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


SVM saved.
SVM
Accuracy: 0.5525
Macro F1 : 0.5477913197913197
Precision: 0.6038928571428572
Recall: 0.5525


/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [ ]:
import torch
from PIL import Image
import pandas as pd
import numpy as np
from tqdm.auto import tqdm
from transformers import CLIPProcessor, CLIPModel
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, f1_score

# === Load CSV and Encode ===
df = pd.read_csv("/content/drive/MyDrive/AML-PROJECT/623final_all.csv")
le = LabelEncoder()
df['label_encoded'] = le.fit_transform(df['label'])

train_df, val_df = train_test_split(df, test_size=0.2, stratify=df['label_encoded'], random_state=42)

# === Load CLIP Model ===
device = 'cuda' if torch.cuda.is_available() else 'cpu'
clip_model = CLIPModel.from_pretrained("openai/clip-vit-base-patch16").to(device)
clip_processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch16")

# === Feature Extraction ===
def extract_features(df):
    feats, labels = [], []
    for _, row in tqdm(df.iterrows(), total=len(df)):
        image = Image.open(row['image_path']).convert("RGB")
        inputs = clip_processor(images=image, return_tensors="pt").to(device)
        with torch.no_grad():
            vision_out = clip_model.vision_model(**inputs)
            pooled = clip_model.visual_projection(vision_out.pooler_output)
        feats.append(pooled.cpu().numpy().squeeze())
        labels.append(row['label_encoded'])
    return np.stack(feats), np.array(labels)

X_train, y_train = extract_features(train_df)
X_val, y_val = extract_features(val_df)

# === Classifier: Logistic Regression ===
clf = LogisticRegression(multi_class='multinomial', max_iter=1000)
clf.fit(X_train, y_train)
y_pred = clf.predict(X_val)
joblib.dump(clf, "/content/drive/MyDrive/AML-PROJECT/logistic/for submission/clip_logistic_noisy.joblib")
print("Logistic model saved.")
# === Evaluation ===
print("logistic")
print("Accuracy:", accuracy_score(y_val, y_pred))
print("Macro F1 :", f1_score(y_val, y_pred, average='macro'))
print("Precision:",precision_score(y_val, y_pred, average='macro'))
print("Recall:",recall_score(y_val, y_pred, average='macro'))




  0%|          | 0/1600 [00:00<?, ?it/s]

  0%|          | 0/400 [00:00<?, ?it/s]

/usr/local/lib/python3.11/dist-packages/sklearn/linear_model/_logistic.py:1247: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(


Logistic model saved.
logistic
Accuracy: 0.32
Macro F1 : 0.27695634920634915
Precision: 0.2798809523809524
Recall: 0.32


/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [ ]:
# 1. Scale features (important for SVM)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_val_scaled = scaler.transform(X_val)

# 1. Scale features (important for SVM)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_val_scaled = scaler.transform(X_val)
# 2. Train SVM
svm = SVC(kernel='linear', C=10, gamma='scale')  # or try kernel='linear'
svm.fit(X_train_scaled, y_train)
y_pred = svm.predict(X_val_scaled)
joblib.dump(svm, "/content/drive/MyDrive/AML-PROJECT/logistic/for submission/clip_svm_noisy.joblib")
print("SVM model saved.")
print("SVM")
# === Evaluation ===
print("Accuracy:", accuracy_score(y_val, y_pred))
print("Macro F1 :", f1_score(y_val, y_pred, average='macro'))
print("Precision:",precision_score(y_val, y_pred, average='macro'))
print("Recall:",recall_score(y_val, y_pred, average='macro'))

SVM model saved.
SVM
Accuracy: 0.2725
Macro F1 : 0.2391150793650794
Precision: 0.24301190476190473
Recall: 0.2725


/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


# dino vit

In [ ]:
import torch
from PIL import Image
import pandas as pd
import numpy as np
from tqdm.auto import tqdm
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, f1_score
from sklearn.model_selection import train_test_split
from transformers import AutoImageProcessor, AutoModel

df = pd.read_csv("/content/drive/MyDrive/AML-PROJECT/623final_all.csv")
le = LabelEncoder()
df['label_encoded'] = le.fit_transform(df['label'])

train_df, val_df = train_test_split(df, test_size=0.2, stratify=df['label_encoded'], random_state=42)
device = 'cuda' if torch.cuda.is_available() else 'cpu'
processor = AutoImageProcessor.from_pretrained("facebook/dinov2-base")
model = AutoModel.from_pretrained("facebook/dinov2-base").to(device).eval()
def extract_features(df):
    feats, labels = [], []
    for _, row in tqdm(df.iterrows(), total=len(df)):
        image = Image.open(row['image_path']).convert("RGB")
        inputs = processor(images=image, return_tensors="pt").to(device)
        with torch.no_grad():
            output = model(**inputs)
            pooled = output.last_hidden_state.mean(dim=1)
        feats.append(pooled.cpu().numpy().squeeze())
        labels.append(row['label_encoded'])
    return np.stack(feats), np.array(labels)

X_train, y_train = extract_features(train_df)
X_val, y_val = extract_features(val_df)

clf = LogisticRegression(multi_class='multinomial', max_iter=1000)
clf.fit(X_train, y_train)
y_pred = clf.predict(X_val)
joblib.dump(clf, "/content/drive/MyDrive/AML-PROJECT/logistic/for submission/dino_log_noisy.joblib")
print("Logistic model saved.")

print("== DINOv2 ViT + Logistic Regression ==")
print("Accuracy:", accuracy_score(y_val, y_pred))
print("Macro F1 :", f1_score(y_val, y_pred, average='macro'))
print(precision_score(y_val, y_pred, average='macro'))
print(recall_score(y_val, y_pred, average='macro'))

preprocessor_config.json:   0%|          | 0.00/436 [00:00<?, ?B/s]

Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.


config.json:   0%|          | 0.00/548 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/346M [00:00<?, ?B/s]

  0%|          | 0/1600 [00:00<?, ?it/s]

  0%|          | 0/400 [00:00<?, ?it/s]

/usr/local/lib/python3.11/dist-packages/sklearn/linear_model/_logistic.py:1247: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(


Logistic model saved.
== DINOv2 ViT + Logistic Regression ==
Accuracy: 0.6925
Macro F1 : 0.6720238095238096
0.7181666666666666
0.6925


/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [ ]:
from sklearn.neural_network import MLPClassifier

clf = MLPClassifier(hidden_layer_sizes=(512,), max_iter=500)
clf.fit(X_train, y_train)
y_pred = clf.predict(X_val)


In [ ]:
joblib.dump(clf, "/content/drive/MyDrive/AML-PROJECT/logistic/for submission/dino_mlp_noisy.joblib")
print("mlp model saved.")

mlp model saved.


In [ ]:
print("== DINOv2 ViT + MLP ==")
print("Accuracy:", accuracy_score(y_val, y_pred))
print("Macro F1 :", f1_score(y_val, y_pred, average='macro'))
print(precision_score(y_val, y_pred, average='macro'))
print(recall_score(y_val, y_pred, average='macro'))

== DINOv2 ViT + MLP ==
Accuracy: 0.665
Macro F1 : 0.6438333333333333
0.6829166666666666
0.665


/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [ ]:
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, f1_score

# 1. Scale features (important for SVM)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_val_scaled = scaler.transform(X_val)

# 2. Train SVM
svm = SVC(kernel='linear', C=10, gamma='scale')  # or try kernel='linear'
svm.fit(X_train_scaled, y_train)
y_pred = svm.predict(X_val_scaled)
joblib.dump(svm, "/content/drive/MyDrive/AML-PROJECT/logistic/for submission/dino_svm_noisy.joblib")
print("Logistic model saved.")
# 3. Evaluate
print("== DINOv2 + SVM ==")
print("Accuracy:", accuracy_score(y_val, y_pred))
print("Macro F1 :", f1_score(y_val, y_pred, average='macro'))
print(precision_score(y_val, y_pred, average='macro'))
print(recall_score(y_val, y_pred, average='macro'))


Logistic model saved.
== DINOv2 + SVM ==
Accuracy: 0.5725
Macro F1 : 0.5483412698412699
0.584845238095238
0.5725


/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


# dino on degrade

In [ ]:
import torch
from transformers import AutoProcessor, AutoModel
from PIL import Image
import pandas as pd
import numpy as np
from tqdm import tqdm
from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import MLPClassifier
from sklearn.svm import SVC
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, f1_score
from sklearn.model_selection import train_test_split

df = pd.read_csv("/content/drive/MyDrive/AML-PROJECT/iris_degrade.csv")
df['ID'] = df['Label'].str.extract(r'(\d+)', expand=False)
id_counts = df['ID'].value_counts()
valid_ids = id_counts[id_counts >= 10].head(100).index
df = df[df['ID'].isin(valid_ids)]

train_df, val_df = train_test_split(df, test_size=0.2, stratify=df['ID'], random_state=42)
le = LabelEncoder()
train_df['encoded_label'] = le.fit_transform(train_df['Label'])
val_df['encoded_label'] = le.transform(val_df['Label'])

device = 'cuda' if torch.cuda.is_available() else 'cpu'
processor = AutoProcessor.from_pretrained("facebook/dinov2-base")
model = AutoModel.from_pretrained("facebook/dinov2-base").to(device)

def extract_features(df, image_key, label_key):
    feats, labels = [], []
    for _, row in tqdm(df.iterrows(), total=len(df)):
        image = Image.open(row[image_key]).convert("RGB")
        inputs = processor(images=image, return_tensors="pt").to(device)
        with torch.no_grad():
            output = model(**inputs)
            pooled = output.last_hidden_state.mean(dim=1).squeeze()
        feats.append(pooled.cpu().numpy())
        labels.append(row[label_key])
    return np.stack(feats), np.array(labels)

X_train, y_train = extract_features(train_df, "Path", "encoded_label")
X_val, y_val = extract_features(val_df, "Path", "encoded_label")

logreg = LogisticRegression(max_iter=1000, multi_class="multinomial")
logreg.fit(X_train, y_train)
y_pred_log = logreg.predict(X_val)
joblib.dump(logreg, "/content/drive/MyDrive/AML-PROJECT/logistic/for submission/dino_log_dg.joblib")
print("Logistic model saved.")
print("== DINOv2 + Logistic Regression ==")
print("Accuracy:", accuracy_score(y_val, y_pred_log))
print("Macro F1 :", f1_score(y_val, y_pred_log, average='macro'))
print(precision_score(y_val, y_pred_log, average='macro'))
print(recall_score(y_val, y_pred_log, average='macro'))

mlp = MLPClassifier(hidden_layer_sizes=(256,), max_iter=300)
mlp.fit(X_train, y_train)
y_pred_mlp = mlp.predict(X_val)
joblib.dump(mlp, "/content/drive/MyDrive/AML-PROJECT/logistic/for submission/dino_mlp_dg.joblib")
print("mlp model saved.")
print("\n== DINOv2 + MLP ==")
print("Accuracy:", accuracy_score(y_val, y_pred_mlp))
print("Macro F1 :", f1_score(y_val, y_pred_mlp, average='macro'))
print(precision_score(y_val, y_pred_mlp, average='macro'))
print(recall_score(y_val, y_pred_mlp, average='macro'))

svm = SVC(kernel='linear', C=10)
svm.fit(X_train, y_train)
y_pred_svm = svm.predict(X_val)
joblib.dump(svm, "/content/drive/MyDrive/AML-PROJECT/logistic/for submission/dino_svm_dg.joblib")
print("svm model saved.")
print("\n== DINOv2 + SVM ==")

print("Accuracy:", accuracy_score(y_val, y_pred_svm))
print("Macro F1 :", f1_score(y_val, y_pred_svm, average='macro'))
print(precision_score(y_val, y_pred_svm, average='macro'))
print(recall_score(y_val, y_pred_svm, average='macro'))

100%|██████████| 400/400 [00:12<00:00, 32.61it/s]
/usr/local/lib/python3.11/dist-packages/sklearn/linear_model/_logistic.py:1247: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(


Logistic model saved.
== DINOv2 + Logistic Regression ==
Accuracy: 0.9225
Macro F1 : 0.9110378510378511
0.9230769230769231
0.926068376068376


/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


mlp model saved.

== DINOv2 + MLP ==
Accuracy: 0.895
Macro F1 : 0.8837874324987727
0.8975085910652921
0.90893470790378


/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


svm model saved.

== DINOv2 + SVM ==
Accuracy: 0.92
Macro F1 : 0.913831615120275
0.9252577319587629
0.9317010309278351


/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
